# Sentiment Analysis với Doc2Vec (Vietnamese)

Word2vec là một khám phá phải nói cực kỳ quan trọng. Nó input là corpus và đầu ra là các Vector cho mỗi từ trong copus đó, với độ dài mỗi vector là cố định, thường khoảng 100-300 chiều. 

Các vector từ này ngoài ra nó còn có tính ngữ nghĩa, và những từ nào tương tự nhau sẽ nằm gần nhau. Nói cách khác, các vector này biểu diễn cách mà ngôn ngữ được tạo ra và cách chúng ta sử dụng chúng mỗi ngày.

Ví dụ `v_man` - `v_woman` xấp xỉ bằng `v_king` - `v_queen`, biểu diễn quan hệ "từ **man** đến **woman** cũng tương tự như **king** đến **queen**".

Quá trình trên trong NLP gọi là **word embedding**. Cách biểu diễn này được sử dụng khá rộng rãi. 

Từ ý tưởng của word2vec, doc2vec lần đầu được giới thiệu để biểu diễn không chỉ mỗi word vector, mà còn biểu diễn được cả câu (sentences) hay đoạn văn (documents). Từ nền tảng word embedding, thử tưởng tượng bạn có thể wector hóa bất kỳ đoạn văn hay câu văn nào vào các vector fixed-length, và áp dụng các thuật toán classification, cluster, ... cơ bản bất kì trên chúng.

Bài này mình sẽ sử dụng Doc2vec để classify data Cornell IMDB movie review corpus (http://www.cs.cornell.edu/people/pabo/movie-review-data/).

## Setup

### Modules

Mình sẽ sử dụng:
- Gensim là thư viện nổi tiếng, implement tốt Word2Vec và Doc2Vec. 
- numpy để thao tác với array.
- sklearn để sử dụng Logistic Regression classifier.

In [1]:
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
import numpy as np

# Classify
from sklearn.linear_model import LogisticRegression

### Input format
Tải dữ liệu từ movie-review-data

> [polarity dataset v2.0](http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz) ( 3.0Mb) (includes README v2.0): 1000 positive and 1000 negative processed reviews. Introduced in Pang/Lee ACL 2004. Released June 2004.

Clean bằng cách chuyển hết về lowercase, xóa tất cả các dấu câu.

In [2]:
! wget -O review_polarity.tar.gz http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz
! tar xzf review_polarity.tar.gz

--2017-04-30 10:12:21--  http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.20
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.20|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3127238 (3,0M) [application/x-gzip]
Saving to: ‘review_polarity.tar.gz’

review_polarity.tar 100%[===================>]   2,98M  2,75MB/s    in 1,1s    

2017-04-30 10:12:22 (2,75 MB/s) - ‘review_polarity.tar.gz’ saved [3127238/3127238]



Data được bỏ vào 2 thư mục pos/ và neg/. Mình sẽ combine, clean và split thành 4 file.

- test-neg.txt: 12500 negative movie reviews from the test data
- test-pos.txt: 12500 positive movie reviews from the test data
- train-neg.txt: 12500 negative movie reviews from the training data
- train-pos.txt: 12500 positive movie reviews from the training data

In [3]:
! head -n 5 txt_sentoken/neg/cv000_29416.txt

plot : two teen couples go to a church party , drink and then drive . 
they get into an accident . 
one of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . 
what's the deal ? 
watch the movie and " sorta " find out . . . 


In [4]:
import shutil
import glob

def combine_files(label):
    with open(label + ".txt", 'w') as outfile:
        for filename in glob.glob('txt_sentoken/%s/*.txt' % label):
            with open(filename, 'rb') as readfile:
                shutil.copyfileobj(readfile, outfile)

combine_files("pos")
combine_files("neg")

In [5]:
# Đếm số dòng
! wc -l *.txt

   31783 neg.txt
   32937 pos.txt
    9535 test-neg.txt
    9881 test-pos.txt
   22247 train-neg.txt
   23055 train-pos.txt
  129438 total


Xóa hết các dấu câu, split thành tập train và test theo tỉ lệ 70/30

In [6]:
import re
import random

def clean_and_split(label):
    with open(label + ".txt", "r") as f:
        content = f.read()
        content = re.sub(r'([^\s\w\n]|_)+', '', content)
        
        # Shuffle data và cắt
        content = content.split('\n')
        random.shuffle(content)
        n_train = int(0.7 * len(content))
        
        content_train = "\n".join(content[:n_train])
        content_test = "\n".join(content[n_train:])
        
        # Lưu xuống file
        with open("train-" + label + ".txt", "wb") as train_f:
            print "Write training data to: %s" % ("train-" + label + ".txt")
            train_f.write(content_train)
            train_f.close()
        with open("test-" + label + ".txt", "wb") as test_f:
            print "Write testing data to: %s" % ("test-" + label + ".txt")
            test_f.write(content_test)
            test_f.close()
            
clean_and_split("pos")
clean_and_split("neg")

Write training data to: train-pos.txt
Write testing data to: test-pos.txt
Write training data to: train-neg.txt
Write testing data to: test-neg.txt


In [7]:
! wc -l *.txt

   31783 neg.txt
   32937 pos.txt
    9535 test-neg.txt
    9881 test-pos.txt
   22247 train-neg.txt
   23055 train-pos.txt
  129438 total


In [8]:
! head -n 4 test-pos.txt

as taran cradles him in his arms  though  gurgi stirshe is not dead after all  
based on the mighty successful musical from broadway  grease was followed in 1980 with the less stellar grease 2  6  510   
disaster is sure to strike  
synopsis  bobby garfield  yelchin  lives in a small town with his mirthless widowed mother  hope davis   


### Feeding Data to Doc2Vec

Doc2Vec nhận input đầu vào là class `LabeledLineSentence`. Bởi về Doc2Vec khác Word2Vec chỗ là nó (Doc2Vec) train cả Doc_Id chung với các word. 

![](https://silvrback.s3.amazonaws.com/uploads/7b02d9b7-20e3-43f8-bed1-e96146611456/sentiment_02_large.png)

Doc2Vec vừa vector hóa từng từ, mà còn xác định vector cả document (đoạn văn) chứa các từ ấy.

Chúng ta phải format data dưới dạng 

```
[['word1', 'word2', 'word3', 'lastword'], ['label1']]
```

In [9]:
class LabeledLineSentence(object):
    def __init__(self, sources):
        self.sources = sources
        
        flipped = {}
        
        # make sure that keys are unique
        for key, value in sources.items():
            if value not in flipped:
                flipped[value] = [key]
            else:
                raise Exception('Non-unique prefix encountered')
    
    def __iter__(self):
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    yield LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no])
    
    def to_array(self):
        self.sentences = []
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    self.sentences.append(LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no]))
        return self.sentences
    
    def total_examples(self):
        return len(self.sentences)
    
    def sentences_perm(self):
        return np.random.permutation(self.sentences)

`LabeledLineSentence` đơn giản sẽ truyền vào 1 `dict`, key là tên file và value là **prefix** của doc_id. Nên đặt prefix khác nhau để tránh lỗi trùng lặp.

In [10]:
sources = {'test-neg.txt':'TEST_NEG', 'test-pos.txt':'TEST_POS', 'train-neg.txt':'TRAIN_NEG', 'train-pos.txt':'TRAIN_POS'}

sentences = LabeledLineSentence(sources)

`sentences` sẽ như sau:

```
[TaggedDocument(words=[u'once', u'mccabe', u'escapes', u'the', u'film', u'becomes', u'the', u'fugitive', u'in', u'reverse', u'and', u'with', u'no', u'thrills'], tags=['TRAIN_NEG_0']),
 TaggedDocument(words=[u'it', u'works', u'as', u'a', u'dry', u'comedy', u'which', u'it', u'does', u'not', u'overplay'], tags=['TRAIN_NEG_1']),
 TaggedDocument(words=[u'you', u'may', u'have', u'noticed', u'that', u'i', u'opted', u'not', u'to', u'describe', u'an', u'iota', u'of', u'mi2s', u'plot'], tags=['TRAIN_NEG_2']), ...
 ```

## Model 

### Building the Vocabulary Table
Doc2vec bắt chúng ta phải xây dựng voca table. `model.build_vocab` sẽ nhận tham số là `LabeledLineSentence`. 

Mình sẽ giải thích sơ các tham số của các dòng code bên dưới:
* min_count: bỏ qua tất cả các từ có tần số xuất hiện nhỏ hơn `min_count`. Chúng ta set = 1, vì mỗi sentence labels chỉ xuất hiện 1 lần, nếu để giá trị cao hơn thì tất cả các labels sẽ bị xóa.
* window: khoảng cách tối đa giữa từ hiện tại và từ predicted. Doc2Vec sử dụng skip-gram model, vậy đây cũng chính là window size của skip-gram model.
* size: Số chiều của vector. 100 là ngon rồi, ai nhiều tài nguyên có thể set lên 300-400.
* sample: threshold để cấu hình higher-frequency words randomly downsampled.
* workers: số worker để training. Nên để bằng số core của cpu.

In [ ]:
model = Doc2Vec(min_count=1, window=10, size=100, sample=1e-4, negative=5, workers=7)
model.build_vocab(sentences.to_array())

### Training Doc2Vec

OK bây giờ bắt đầu train model. Model sẽ càng tốt nếu chúng ta train nhiều lần và
mỗi lần thứ tự các sentences là khác nhau - hàm `sentences_perm()` của `LabeledLineSentences` có tác dụng đó.

Chúng ta train đi train lại khoảng 10 lần. Ai có thời gian thì 20 hoặc 30.
Tốn khoảng 10 phút.

In [ ]:
model.train(sentences.sentences_perm(), total_examples=sentences.total_examples(), epochs=5)

Exception in thread Thread-11:
Traceback (most recent call last):
  File "/usr/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/usr/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/home/lvduit08/.local/lib/python2.7/site-packages/gensim/models/word2vec.py", line 853, in job_producer
    sentence_length = self._raw_word_count([sentence])
  File "/home/lvduit08/.local/lib/python2.7/site-packages/gensim/models/doc2vec.py", line 722, in _raw_word_count
    return sum(len(sentence.words) for sentence in job)
  File "/home/lvduit08/.local/lib/python2.7/site-packages/gensim/models/doc2vec.py", line 722, in <genexpr>
    return sum(len(sentence.words) for sentence in job)
AttributeError: 'numpy.ndarray' object has no attribute 'words'

